# Multi-Plane Imaging Example

This notebook demonstrates how to use the ndx-microscopy extension for storing and analyzing multi-plane imaging data in NWB format. Multi-plane imaging differs from both planar and volumetric imaging in several key aspects:

- Requires separate imaging spaces and series for each plane
- Uses specialized containers to group related data from different planes

## Import Required Libraries

Import necessary packages from NWB, ndx-microscopy, and ndx-ophys-devices extensions, along with other utility libraries. Note the additional imports specific to multi-plane imaging like MultiPlaneMicroscopyContainer and SegmentationContainer.

In [ ]:
from datetime import datetime
from uuid import uuid4
import numpy as np
from pynwb import NWBFile, NWBHDF5IO

from ndx_microscopy import (
    MicroscopeModel,
    Microscope,
    MicroscopyRig,
    LineScan,
    MicroscopyChannel,
    PlanarImagingSpace,
    PlanarMicroscopySeries,
    MultiPlaneMicroscopyContainer,
    PlanarSegmentation,
    SummaryImage,
    MicroscopyResponseSeries,
    MicroscopyResponseSeriesContainer,
    SegmentationContainer
)

from ndx_ophys_devices import (
    ExcitationSourceModel,
    PulsedExcitationSource,
    BandOpticalFilterModel,
    BandOpticalFilter,
    DichroicMirrorModel,
    DichroicMirror,
    PhotodetectorModel,
    Photodetector,
    Indicator
)

## Create NWB File

Initialize a new NWB file with basic session information for multi-plane imaging.

In [ ]:
nwbfile = NWBFile(
    session_description='Multi-plane imaging session',
    identifier=str(uuid4()),
    session_start_time=datetime.now(),
    lab='Neural Circuits Lab',
    institution='University of Neuroscience',
    experiment_description='Multi-plane imaging with ETL'
)

## Set Up Microscope and Components

Configure the microscope and its various components, including the electrically tunable lens (ETL) for multi-plane imaging:
- Microscope model and instance with ETL
- Laser source for deep tissue penetration
- Optical components (filters and dichroic)
- PMT detector
- GCaMP6f indicator

In [ ]:
# Set up microscope model
microscope_model = MicroscopeModel(
    name='etl-model',
    description='Two-photon microscope model with electrically tunable lens',
    model_number='etl-001',
    manufacturer='Custom Build'
)
nwbfile.add_device(microscope_model)

# Set up microscope with ETL technique
microscope = Microscope(
    name='etl-scope',
    description='Two-photon microscope with electrically tunable lens',
    serial_number='etl-serial-001',
    model=microscope_model,
    technique='electrically tunable lens'
)
nwbfile.add_device(microscope)

In [ ]:
# Configure laser model and instance
excitation_source_model = ExcitationSourceModel(
    name="chameleon_model",
    manufacturer="Coherent",
    model_number="Chameleon Ultra II",
    description="Excitation source model for multi-plane imaging",
    source_type="laser",
    excitation_mode="two-photon",
    wavelength_range_in_nm=[800.0, 1000.0]
)
nwbfile.add_device(excitation_source_model)

laser = PulsedExcitationSource(
    name='laser',
    description='Femtosecond pulsed laser for multi-plane imaging',
    serial_number="CU2-SN-123456",
    model=excitation_source_model,
    power_in_W=1.5,
    peak_power_in_W=100000.0,
    peak_pulse_energy_in_J=1.25e-9,
    pulse_rate_in_Hz=80.0e6,
)
nwbfile.add_device(laser)

In [ ]:
# Configure optical components
excitation_filter_model = BandOpticalFilterModel(
    name="excitation_filter_model",
    filter_type="Bandpass",
    manufacturer="Semrock",
    model_number="FF01-920/80",
    center_wavelength_in_nm=920.0,
    bandwidth_in_nm=80.0
)
nwbfile.add_device(excitation_filter_model)

excitation_filter = BandOpticalFilter(
    name='excitation_filter',
    description='Excitation filter for multi-plane imaging',
    serial_number="EF-SN-123456",
    model=excitation_filter_model
)
nwbfile.add_device(excitation_filter)

dichroic_mirror_model = DichroicMirrorModel(
    name="primary_dichroic_model",
    manufacturer="Semrock",
    model_number="FF757-Di01",
    cut_on_wavelength_in_nm=757.0,
    cut_off_wavelength_in_nm=750.0,
    transmission_band_in_nm=[757.0, 1100.0],
    reflection_band_in_nm=(400.0, 750.0),
    angle_of_incidence_in_degrees=45.0
)
nwbfile.add_device(dichroic_mirror_model)

dichroic = DichroicMirror(
    name="primary_dichroic",
    description="Dichroic mirror for multi-plane imaging",
    serial_number="DM-SN-123456",
    model=dichroic_mirror_model
)
nwbfile.add_device(dichroic)

emission_filter_model = BandOpticalFilterModel(
    name="emission_filter_model",
    filter_type="Bandpass",
    manufacturer="Semrock",
    model_number="FF01-510/84",
    center_wavelength_in_nm=510.0,
    bandwidth_in_nm=84.0
)
nwbfile.add_device(emission_filter_model)

emission_filter = BandOpticalFilter(
    name='emission_filter',
    description='Emission filter for GCaMP6f',
    serial_number="EF-SN-123456",
    model=emission_filter_model
)
nwbfile.add_device(emission_filter)

photodetector_model = PhotodetectorModel(
    name="pmt_model",
    detector_type="PMT",
    manufacturer="Hamamatsu",
    model_number="R6357",
    gain=70.0,
    gain_unit="dB"
)
nwbfile.add_device(photodetector_model)

detector = Photodetector(
    name='pmt',
    description='PMT detector for fluorescence detection',
    serial_number="PMT-SN-123456",
    model=photodetector_model
)
nwbfile.add_device(detector)

In [ ]:
# Create indicator
indicator = Indicator(
    name='gcamp6f',
    label='GCaMP6f',
    description='Calcium indicator for multi-plane imaging',
    manufacturer='Addgene',
    injection_brain_region='Visual cortex',
    injection_coordinates_in_mm=[-2.5, 3.2, 0.5]
)

## Configure Microscopy Rig

Set up the microscopy rig, connecting all the previously defined components. The ETL is part of the excitation path, enabling rapid focusing at different depths.

In [ ]:
# Create microscopy rig
microscopy_rig = MicroscopyRig(
    name='etl_rig',
    description='Multi-plane microscopy rig with ETL',
    microscope=microscope,
    excitation_source=laser,
    excitation_filter=excitation_filter,
    dichroic_mirror=dichroic,
    photodetector=detector,
    emission_filter=emission_filter
)

# Create microscopy channel
microscopy_channel = MicroscopyChannel(
    name='gcamp_channel',
    description='GCaMP6f channel',
    excitation_wavelength_in_nm=920.0,
    emission_wavelength_in_nm=510.0,
    indicator=indicator
)

## Create Multiple Imaging Planes

Set up multiple imaging planes at different depths. For each plane:
1. Create a planar imaging space
2. Generate example imaging data
3. Create a planar microscopy series
4. Set up ROI segmentation
5. Calculate ROI responses

The planes are later grouped into containers for organization.

In [ ]:
# Create ophys processing module
ophys_module = nwbfile.create_processing_module(
    name='ophys',
    description='Optical physiology processing module'
)

# Initialize lists for collecting plane-specific data
planar_series_list = []
segmentation_list = []
response_series_list = []
depths = [-100, -50, 0, 50, 100]  # Depths in µm

illumination_pattern = LineScan(
    name="line_scanning",
    description="Line scanning two-photon microscopy",
    scan_direction="horizontal",
    line_rate_in_Hz=1000.0,
    dwell_time_in_s=1.0e-6,
)

for depth in depths:
    # Create example data for this plane
    frames = 1000
    height = 512
    width = 512
    data = np.random.rand(frames, height, width)
    
    # Create imaging space for this depth
    plane_space = PlanarImagingSpace(
        name=f"plane_depth_{depth}",
        description=f"Imaging plane at {depth} µm depth",
        pixel_size_in_um=[1.0, 1.0],
        dimensions_in_pixels=[height, width],
        origin_coordinates=[-1.2, -0.6, depth / 1000],  # Convert to mm
        location="Visual cortex",
        reference_frame="bregma",
        orientation="RAS",
        illumination_pattern=illumination_pattern,
    )

    # Create imaging series for this plane
    plane_series = PlanarMicroscopySeries(
        name=f'imaging_depth_{depth}',
        description=f'Imaging data at {depth} µm depth',
        microscopy_channel=microscopy_channel,
        microscopy_rig=microscopy_rig,
        planar_imaging_space=plane_space,
        data=data,
        unit='a.u.',
        conversion=1.0,
        offset=0.0,
        rate=30.0,
        starting_time=0.0
    )
    planar_series_list.append(plane_series)

    # Create summary images for this plane
    mean_image = SummaryImage(
        name=f"mean_{depth}", description=f"Mean intensity projection at {depth} µm", data=np.mean(data, axis=0)
    )

    max_image = SummaryImage(
        name=f"max_{depth}", description=f"Maximum intensity projection at {depth} µm", data=np.max(data, axis=0)
    )

    # Create segmentation for this plane
    segmentation = PlanarSegmentation(
        name=f"rois_{depth}",
        description=f"ROI segmentation at {depth} µm",
        planar_imaging_space=plane_space,
        summary_images=[mean_image, max_image],
    )

    # Add ROIs
    roi_mask = np.zeros((height, width), dtype=bool)
    roi_mask[256:266, 256:266] = True
    segmentation.add_roi(image_mask=roi_mask)

    segmentation_list.append(segmentation)

    # Create ROI responses
    roi_region = segmentation.create_roi_table_region(
        description=f"ROIs at {depth} µm", region=list(range(len(segmentation.id)))
    )

    # Extract responses
    num_rois = len(segmentation.id)
    responses = np.zeros((frames, num_rois))

    for i, roi_mask in enumerate(segmentation.image_mask[:]):
        roi_data = data[:, roi_mask]
        responses[:, i] = np.mean(roi_data, axis=1)

    # Create response series
    response_series = MicroscopyResponseSeries(
        name=f"responses_{depth}",
        description=f"Fluorescence responses at {depth} µm",
        data=responses,
        rois=roi_region,
        unit="n.a.",
        rate=30.0,
        starting_time=0.0,
        microscopy_series=plane_series,
    )
    response_series_list.append(response_series)

## Create Data Containers

Group related data from different planes into specialized containers:
- MultiPlaneMicroscopyContainer for raw imaging data
- SegmentationContainer for ROI definitions
- MicroscopyResponseSeriesContainer for ROI responses

In [ ]:
# Create containers
multi_plane_container = MultiPlaneMicroscopyContainer(
    name='multi_plane_data',
    planar_microscopy_series=planar_series_list
)
nwbfile.add_acquisition(multi_plane_container)

segmentation_container = SegmentationContainer(
    name='plane_segmentations',
    segmentations=segmentation_list
)
ophys_module.add(segmentation_container)

response_container = MicroscopyResponseSeriesContainer(
    name='plane_responses',
    microscopy_response_series=response_series_list
)
ophys_module.add(response_container)

## Save and Load NWB File

Write the NWB file to disk and demonstrate how to read back multi-plane data, accessing specific planes and their associated ROIs and responses.

In [9]:
# Save file
with NWBHDF5IO('multi_plane_imaging.nwb', 'w') as io:
    io.write(nwbfile)

# Read file and access data
with NWBHDF5IO('multi_plane_imaging.nwb', 'r') as io:
    nwbfile = io.read()
    
    # Access multi-plane data
    multi_plane = nwbfile.acquisition['multi_plane_data']
    
    # Access specific plane data
    plane_0 = multi_plane.planar_microscopy_series['imaging_depth_0']
    plane_data = plane_0.data[:]
    
    # Access ROI data
    ophys = nwbfile.processing['ophys']
    segmentations = ophys['plane_segmentations']
    rois_0 = segmentations['rois_0']
    roi_masks = rois_0.image_mask[:]
    
    # Access responses
    responses = ophys['plane_responses']
    responses_0 = responses['responses_0']
    roi_data = responses_0.data[:]